Die Bewegungsgleichung für 3 oder mehr Körper lässt sich im Allgemeinen nicht analytisch lösen. Numerisch gibt es verschieden anstäze:

#### Trivial Euler Verfahren:

Man wählt einen kleinen Zeitschritt $dt$, berechnet dann mit der Geschwindigkeit $v$ daraus die Koordinaten $r(t+dt)=r(t)+dt*v(t)$ für den nächsten Schritt und aus den Koordinaten $r$ die Kraft $F(r)$ und damit die Beschleunigung $a=F/m$ auf den Körper und passt daraus die Geschwindigkeit $v(t+dt)=v(t)+a(t+dt)*dt$ für den nächsten Zeitschritt an, um die Berechnung für diesen zu wiederholen. 
Dieses verfahren ist aber sehr instabil, denn selbst bei einer konstanten Beschleunigung ist die Geschwindigkeit mit der bei einem Zeitschritt gerechnet wird immer nur für den Anfangszeitpunkt des Zeitschrittes exakt und danach immer zu wenig in die beschleunigte Richtung. Für eine konstante Kraft kommt so in jedem Zeitschritt der gleiche Fehler hinzu und der summiert sich auf und wird immer größer. Beim Beispiel der Planetenbewegung ist die Kraft zwar nicht konstant, aber immer nach innen gerichtet, sodass der Planet bei dem Verfahren immer nicht weit genug nach innen beschleunigt wird, und immer weiter nach außen wegdriftet.

#### Besser Verlet Verfahren:

Vor dem ersten Schritt berechnet man die Kraft $F$ und die Beschleunigung $a=F/m$ auf den Körper entsprechend der Startkoordinaten, aber passt die Geschwindigkeit nur um die Beschleunigung für einen halben Zeitschritt an $v(t+½*dt) = v(t)+½*dt*a(t)$. Dann werden in jedem Zeitschritt die Koordinaten $r(t+dt)=r(t)+v(t+½*dt)*dt$ anhand der Geschwindigkeit zum Zeitpunkt in der Mitte des Zeitschrittes berechnet. Und die Geschwindigkeit $v(t+3/2*dt)=v(t+½*dt)+a(t+dt)*dt$ wird anhand der Beschleunigung aus der Kraft zu den Koordinaten von ganzzahligen Zeitschritten berechnet, wobei das wieder die Beschleunigung zu dem Zeitpunkt ist, der genau in der Mitte zwischen den Zeitpunkten liegt, für die die Geschwindigkeiten berechnet wurden.
Also wäre die Rechnung schonmal für sich linear mit der Zeit ändernde Beschleunigungen und Geschwindigkeiten exakt und der Rechenaufwand ist kaum gestiegen
Das Verlet verfahren lässt sich auch über eine Taylorentwicklung zur 3. Ordnung von der Mitte in beide Richtungen herleiten, der Fehler liegt damit erst in 4. Ordnung. Die antisymmetrische Anordnung des 1. und 3. Terms lässt sich diese wegkürzen, sodass der Rechenaufwand niedrig bleibt.
Außerdem bleibt beim verlet-Verfahren das Volumen des Phasenraums erhalten.


solar.py:

In [ ]:
gravConst = 6.6743 * (10**(-11)) 
mass_sun = 1.989 * (10**30) * gravConst
mass_mercury = 3.285 * (10**23)  * gravConst
mass_venus = 4.867 * (10**24)  * gravConst
mass_earth = 5.972 * (10**24)  * gravConst
mass_mars = 6.39 * (10**23)  * gravConst
mass_jupiter = 19000 * (10**23) * gravConst
mass_saturn = 5690 * (10**23) * gravConst
mass_uranus = 870 * (10**23) * gravConst
mass_neptune = 1.024 * (10**26) * gravConst
mass_moon = 7.342 * (10**22)  * gravConst
distance_mercury = 63.867 * (10**9) 
distance_venus = 108.84 * (10**9)
distance_earth = 152.1 * (10**9)
distance_mars = 230.66 * (10**9)
distance_jupiter = 778.3 * (10**9)
distance_saturn = 1427 * (10**9)
distance_uranus = 2870 * (10**9)
distance_neptune = 4496 * (10**9)
distance_moon = 384.4 * (10**6)
velocity_mercury = 47870
velocity_venus = 35020
velocity_earth = 29300
velocity_mars = 24077
velocity_jupiter = 13070
velocity_saturn = 9650
velocity_uranus = 6810
velocity_neptune = 544
velocity_moon = 1022

earth_year = 60 * 60 * 24* 365.2

## Einheiten

Beim verlet-Algorithmus werden für $n$ Iterationen mit dem Zeitschritt $h$ Position $\vec{r}$ und Geschwindigkeit $\vec{v}$ folgender Zeitpunkte ausgerechnet:
$$ \vec{r}(t_0 + h*i) $$
$$ \vec{v}(t_0 - \frac{h}{2} + h*i) $$
wobei
$$ i \in {1,...,n} $$
Die Werte für $\vec{r}(t)$ und $\vec{v}(t)$ werden dabei in folgendermaßen berechnet:
$$ \vec{r}(t) = \vec{r}(t-h) + h*\vec{v}(t-\frac{h}{2})$$
$$ \vec{v}(t) = \vec{v}(t-h) + h*\vec{a}(t-\frac{h}{2})$$
Dabei ist $\vec{a}(t)$ die Beschleunigung, die aus $\vec{r}(t)$ folgendermaßen berechnet wird:
$$ \vec{a} = \frac{\vec{F}}{m} $$
$$ \vec{F} = G*m*\sum_{j=1}^p\frac{m_j}{|\vec{r_j}-r|^2}$$
$$ \Rightarrow \vec{a} = G*\sum_{j=1}^p\frac{m_j}{|\vec{r_j}-r|^2}$$
Dabei sind $m_j$ die Massen und $\vec{r_j}$ die Ortsvektoren der $p$ anderen Körper im System.
Um das ständige Multiplizieren mit der Gravitationskonstante $G$ wärend der Simulation zu vermeiden, können wir stattdessen einfach zu Beginn alle Massen jeweis mit ihr multiplizieren und diese Größe statt der Masse unter particle.mass speichern. Dann berechnet sich die Beschleunigung folgendermaßen:
$$ \vec{a} = \sum_{j=1}^p\frac{\lambda_j}{|\vec{r_j}-r|^2}$$
mit $$ \lambda_j = m_j*G $$
Dabei sind $\lambda_j$ die mit der Gravitationskonstante multiplizierten massen particle.mass.
Die Massen werden im Code aber auch für die Berechnung der kinetischen und potentiellen Energie verwendet, also muss bei diesen Berechnungen die Masse durch $\frac{\lambda}{G}$ ersetzt werden:
$$ W_{pot}=G\frac{-m_1*m_2}{|\vec{r_1}-\vec{r_2}|} = \frac{1}{G}*\frac{-\lambda_1*\lambda_2}{|\vec{r_1}-\vec{r_2}|}$$
$$ W_{kin}=\frac{1}{2}*m*v^2 = \frac{1}{G}*\lambda*v^2$$
Bei diesen Formeln sieht man dann, dass man die Gravitationskonstante einfach weglassen kann, und alle Energien werden mit dem gleichen Faktor skaliert, sodass alle Verhältnisse noch Stimmen.